# Ciclos Políticos do Ceará

**Trabalho de Conclusão de Curso — MBA em Jornalismo de Dados (IDP)** 
**Responsável:** Igor Cavalcante — victorigor.ac@gmail.com  
**Tema:** Análise dos ciclos políticos do Ceará ao longo de 30 anos  
**Etapa:** 3/3 — Visualizações
**Data:** 11/2025  

Este notebook é o terceiro de três que analisam os ciclos políticos no Ceará nas últimas três décadas. A partir do desempenho partidário, a pesquisa mapeia as legendas que cresceram e reduziram no Estado, e fornecer dados para uma reportagem que explique o que há por trás dessas oscilações.  

Para tanto, serão considerados os desempenhos partidários de prefeitos, deputados estaduais e federais. No caso dos mandatários municipais, eles revelam quem controla a política local e os recursos municipais. Já os deputados estaduais consolidam o apoio político ao Governo do Estado. E os deputados federais mostram como o Ceará se insere nas dinâmicas nacionais dos partidos.  

Neste terceiro momento, os dados coletados a partir do [portal de Dados Abertos do Tribunal Superior Eleitoral](https://dadosabertos.tse.jus.br/), padronizados e analisados serão consolidados em visualizações que farão parte da reportagem.

Para a pesquisa, foram considerados os dados de Resultado das eleições municipais e gerais entre 1994 e 2024, totalizando 16 pleitos.

### Configuração inicial

Estrutura a análise conectando os arquivos processados anteriormente e definindo variáveis.

In [38]:
# Bibliotecas, caminhos, bloco de leituras e definições
import os
from pathlib import Path
import pandas as pd

pasta_base = r"C:\Python\eleicoes_ceara\br_dados_processados"
prefeitos_coef = fr"{pasta_base}\coeficientes_prefeitos_BR_CE.csv"
deputados_coef = fr"{pasta_base}\coeficientes_deputados_BR_CE.csv"

df_prefeitos = pd.read_csv(prefeitos_coef)
df_deputados = pd.read_csv(deputados_coef)

# Partidos mais relevantes no Estado - Série histórica

Compara os desempenhos proporcionais das principais siglas (que tiveram 25% ou mais de representação em algum pleito) no Ceará.

### Função

Seleção de partidos relevantes a partir dos coeficientes já calculados de prefeitos e deputados. 

Critério: coeficiente mínimo atingido em representação em alguma eleição na série histórica. O código prevê que o critério pode ser alterado a partir da análise a ser feita. 

In [39]:
def definir_relevancia(df, criterio):
    df = df[["ANO_ELEICAO", "SG_PARTIDO", "coeficiente_CE"]].copy()
    
    # Identifica partidos com coeficiente mínimo em qualquer ano
    partidos_relevantes = df[df["coeficiente_CE"] >= criterio]["SG_PARTIDO"].unique()
    df_filtrado = df[df["SG_PARTIDO"].isin(partidos_relevantes)]
    
    # Garante que todos os partidos apareçam em todos os anos
    anos = sorted(df_filtrado["ANO_ELEICAO"].unique())
    partidos = sorted(df_filtrado["SG_PARTIDO"].unique())
    idx = pd.MultiIndex.from_product([anos, partidos], names=["ANO_ELEICAO", "SG_PARTIDO"])
    df_completo = (
        df_filtrado.set_index(["ANO_ELEICAO", "SG_PARTIDO"])
        .reindex(idx, fill_value=0)
        .reset_index()
    )
    
    pivot = df_completo.pivot(index="ANO_ELEICAO", columns="SG_PARTIDO", values="coeficiente_CE").reset_index()
    return pivot

In [40]:
def definir_relevancia(df, criterio):
    # Identifica partidos com coeficiente mínimo em qualquer ano
    partidos_relevantes = df[df["coeficiente_CE"] >= criterio]["SG_PARTIDO"].unique()
    df_filtrado = df[df["SG_PARTIDO"].isin(partidos_relevantes)]
    
    # Garante que todos os partidos apareçam em todos os anos
    anos = sorted(df_filtrado["ANO_ELEICAO"].unique())
    partidos = sorted(df_filtrado["SG_PARTIDO"].unique())
    idx = pd.MultiIndex.from_product([anos, partidos], names=["ANO_ELEICAO", "SG_PARTIDO"])
    df_completo = (
        df_filtrado.set_index(["ANO_ELEICAO", "SG_PARTIDO"])
        .reindex(idx, fill_value=0)
        .reset_index()
    )
    return df_completo

def pivot(df):
    df = df[["ANO_ELEICAO", "SG_PARTIDO", "coeficiente_CE"]].copy()
    df = df.pivot(index="ANO_ELEICAO", columns="SG_PARTIDO", values="coeficiente_CE").fillna(0).reset_index()
    return df

### Deputados estaduais e federais

> Critério para 25% como coeficiente mínimo em alguma eleição da série histórica.

In [41]:
dep_fed = df_deputados[df_deputados["DS_CARGO"] == "DEPUTADO FEDERAL"]
dep_est = df_deputados[df_deputados["DS_CARGO"] == "DEPUTADO ESTADUAL"]

criterio = 25

dep_fed_selecionados = definir_relevancia(dep_fed, criterio)
dep_fed_selecionados = pivot(dep_fed_selecionados)

dep_est_selecionados = definir_relevancia(dep_est, criterio)
dep_est_selecionados = pivot(dep_est_selecionados)

dep_fed_selecionados.to_csv(pasta_base + rf"\selecionados_deputados_federal_{criterio}_CE.csv", index=False)
dep_est_selecionados.to_csv(pasta_base + rf"\selecionados_deputados_estadual_{criterio}_CE.csv", index=False)

print(pasta_base + rf"\selecionados_deputados_federal_{criterio}_CE.csv")
print(pasta_base + rf"\selecionados_deputados_estadual_{criterio}_CE.csv")

C:\Python\eleicoes_ceara\br_dados_processados\selecionados_deputados_federal_25_CE.csv
C:\Python\eleicoes_ceara\br_dados_processados\selecionados_deputados_estadual_25_CE.csv


> Critério para 5% como coeficiente mínimo em alguma eleição da série histórica.

In [42]:
criterio = 5

dep_fed_selecionados = definir_relevancia(dep_fed, criterio)

dep_est_selecionados = definir_relevancia(dep_est, criterio)

dep_fed_selecionados.to_csv(pasta_base + rf"\selecionados_deputados_federal_{criterio}_CE.csv", index=False)
dep_est_selecionados.to_csv(pasta_base + rf"\selecionados_deputados_estadual_{criterio}_CE.csv", index=False)

print(pasta_base + rf"\selecionados_deputados_federal_{criterio}_CE.csv")
print(pasta_base + rf"\selecionados_deputados_estadual_{criterio}_CE.csv")

C:\Python\eleicoes_ceara\br_dados_processados\selecionados_deputados_federal_5_CE.csv
C:\Python\eleicoes_ceara\br_dados_processados\selecionados_deputados_estadual_5_CE.csv


### Prefeitos

> Critério para 25% como coeficiente mínimo em alguma eleição da série histórica.

In [43]:
criterio = 25

prefeitos_selecionados = definir_relevancia(df_prefeitos, criterio)
prefeitos_selecionados = pivot(prefeitos_selecionados)

prefeitos_selecionados.to_csv(pasta_base + rf"\selecionados_prefeitos_{criterio}_CE.csv", index=False)
print(pasta_base + rf"\selecionados_prefeitos_{criterio}_CE.csv")

C:\Python\eleicoes_ceara\br_dados_processados\selecionados_prefeitos_25_CE.csv


> Critério para 5% como coeficiente mínimo em alguma eleição da série histórica.

In [44]:
criterio = 5

prefeitos_selecionados = definir_relevancia(df_prefeitos, criterio)

prefeitos_selecionados.to_csv(pasta_base + rf"\selecionados_prefeitos_{criterio}_CE.csv", index=False)
print(pasta_base + rf"\selecionados_prefeitos_{criterio}_CE.csv")

C:\Python\eleicoes_ceara\br_dados_processados\selecionados_prefeitos_5_CE.csv


# Lulismo vs Bolsonarismo

Medindo o impacto da polarização no País nas siglas lulistas e bolsonaristas do Ceará.

In [48]:
df_prefeitos_lulismo_bolsonarismo = df_prefeitos[df_prefeitos['SG_PARTIDO'].isin(['PT', 'PSL', 'PL*'])]
df_prefeitos_lulismo_bolsonarismo = pivot(df_prefeitos_lulismo_bolsonarismo)
df_prefeitos_lulismo_bolsonarismo.to_csv(pasta_base + rf"\df_prefeitos_lulismo_bolsonarismo.csv", index=False)
print(pasta_base + rf"\df_prefeitos_lulismo_bolsonarismo.csv")

C:\Python\eleicoes_ceara\br_dados_processados\df_prefeitos_lulismo_bolsonarismo.csv


In [51]:
df_deputados_lulismo_bolsonarismo = df_deputados[df_deputados['SG_PARTIDO'].isin(['PT', 'PSL', 'PL*'])]
df_deputado_ESTADUAL_lulismo_bolsonarismo= df_deputados_lulismo_bolsonarismo[df_deputados_lulismo_bolsonarismo["DS_CARGO"] == "DEPUTADO ESTADUAL"]
df_deputados_FEDERAL_lulismo_bolsonarismo = df_deputados_lulismo_bolsonarismo[df_deputados_lulismo_bolsonarismo["DS_CARGO"] == "DEPUTADO FEDERAL"]

df_deputado_ESTADUAL_lulismo_bolsonarismo = pivot(df_deputado_ESTADUAL_lulismo_bolsonarismo)
df_deputado_ESTADUAL_lulismo_bolsonarismo.to_csv(pasta_base + rf"\df_deputado_ESTADUAL_lulismo_bolsonarismo.csv", index=False)

df_deputados_FEDERAL_lulismo_bolsonarismo = pivot(df_deputados_FEDERAL_lulismo_bolsonarismo)
df_deputados_FEDERAL_lulismo_bolsonarismo.to_csv(pasta_base + rf"\df_deputados_FEDERAL_lulismo_bolsonarismo.csv", index=False)

print(pasta_base + rf"\df_deputado_ESTADUAL_lulismo_bolsonarismo.csv")
print(pasta_base + rf"\df_deputados_FEDERAL_lulismo_bolsonarismo.csv")

C:\Python\eleicoes_ceara\br_dados_processados\df_deputado_ESTADUAL_lulismo_bolsonarismo.csv
C:\Python\eleicoes_ceara\br_dados_processados\df_deputados_FEDERAL_lulismo_bolsonarismo.csv
